# Time Series Pipelines

Pipelines let you **chain preprocessing + modeling + evaluation** in a reproducible way, preventing leakage and enabling fair comparisons.

See `01_sktime_pipelines` for full pipeline patterns, backtesting, and ML reduction workflows.


In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sktime.datasets import load_airline

# Reproducibility
np.random.seed(42)

y = load_airline()
y.name = "Passengers"



In [ ]:
from sktime.transformations.series.impute import Imputer
from sktime.transformations.series.detrend import Detrender
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.forecasting.model_selection import temporal_train_test_split, ForecastingHorizon

pipe = TransformedTargetForecaster([
    ("impute", Imputer(method="mean")),
    ("detrend", Detrender()),
    ("model", NaiveForecaster(strategy="last")),
])

train, test = temporal_train_test_split(y, test_size=24)
fh = ForecastingHorizon(test.index, is_relative=False)
pipe.fit(train)
pred = pipe.predict(fh)



## Visualization


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index.to_timestamp(), y=train, name="Train"))
fig.add_trace(go.Scatter(x=test.index.to_timestamp(), y=test, name="Test"))
fig.add_trace(go.Scatter(x=pred.index.to_timestamp(), y=pred, name="Forecast"))
fig.update_layout(title="Pipeline forecast vs actual")
fig.show()
